In [1]:
import numpy      as np
import pandas     as pd
import seaborn    as sns
from sklearn.model_selection   import train_test_split
import matplotlib.pyplot as plt
import missingno as msno

In [2]:
hospital_male_frm = pd.read_csv("./hospital_male_test.csv", encoding='cp949')
hospital_female_frm =  pd.read_csv("./hospital_female_test.csv", encoding='cp949')

In [3]:
# hospital_male_frm["rhm_min_max_dif"] = (hospital_male_frm["avg_rhm"] - hospital_male_frm["min_rhm"]) * 2
# hospital_female_frm["rhm_min_max_dif"] = (hospital_female_frm["avg_rhm"] - hospital_female_frm["min_rhm"])*2
hospital_male_frm["rhm_min_avg_dif"] = (hospital_male_frm["avg_rhm"] - hospital_male_frm["min_rhm"])
hospital_female_frm["rhm_min_avg_dif"] = (hospital_female_frm["avg_rhm"] - hospital_female_frm["min_rhm"])

In [4]:
hospital_male_frm['min_max_Ta'] = hospital_male_frm['max_ta'] - hospital_male_frm['min_ta']
hospital_female_frm['min_max_Ta'] = hospital_female_frm['max_ta'] - hospital_female_frm['min_ta']

In [5]:
hospital_male_frm['min_max_ps'] = hospital_male_frm['max_ps'] - hospital_male_frm['min_ps']
hospital_female_frm['min_max_ps'] = hospital_female_frm['max_ps'] - hospital_female_frm['min_ps']

In [6]:
# 지역 인구대비 질환자 발생비율 컬럼. 10**6을 한 이유는 광역시, 도 단위가 100만 단위로 떨어지기때문에 eda를 위해 곱함
hospital_male_frm["D/R"] = (hospital_male_frm["frequency"]/hospital_male_frm["total_pop"])*(10**6)
hospital_female_frm["D/R"] = (hospital_female_frm["frequency"]/hospital_female_frm["total_pop"])*(10**6)

In [7]:
display(hospital_female_frm[["area","tma","D/R"]])
print()
display(hospital_male_frm[["area","tma","D/R"]])

,area,tma,D/R
0,강원,2011-12-01,3.930972
1,경기,2011-12-01,0.844301
2,경남,2011-12-01,0.608269
3,경북,2011-12-01,0.744342
4,광주,2011-12-01,2.710549
...,...,...,...
31581,전남,2016-12-31,NaN
31582,전북,2016-12-31,NaN
31583,제주,2016-12-31,NaN
31584,충남,2016-12-31,NaN


,area,tma,D/R
0,강원,2011-12-01,3.879588
1,경기,2011-12-01,0.664965
2,경남,2011-12-01,1.201378
3,경북,2011-12-01,4.425670
4,광주,2011-12-01,0.000000
...,...,...,...
31581,전남,2016-12-31,NaN
31582,전북,2016-12-31,NaN
31583,제주,2016-12-31,NaN
31584,충남,2016-12-31,NaN


In [8]:
#질환발생자 없는 날 확인
fail1_frm = hospital_male_frm[hospital_male_frm["D/R"]<=0.001]
fail2_frm = hospital_female_frm[hospital_female_frm["D/R"]<=0.001]
print()
display(fail1_frm[["D/R"]])
print()
display(fail2_frm[["D/R"]])

,D/R
4,0.0
9,0.0
10,0.0
14,0.0
15,0.0
...,...
24829,0.0
24830,0.0
24832,0.0
24834,0.0


,D/R
5,0.0
9,0.0
10,0.0
14,0.0
17,0.0
...,...
24831,0.0
24833,0.0
24834,0.0
24835,0.0


In [9]:
hospital_male_frm.to_csv('./hospital_male_0726_test.csv', encoding='cp949')

In [10]:
hospital_female_frm.to_csv('./hospital_female_0726_test.csv', encoding='cp949')